# Deploy <font color='red'>StudioAI Plan Recommendation Model</font> Model Package from AWS Marketplace 




<font color='red'>Overview</font>

This sample notebook shows you how to deploy <font color='red'>StudioAI Plan Recommendation Model</font> using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess** and **AmazonS3FullAccess** 
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to <font color='red'>StudioAI Plan Recommendation Model</font>. 

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'>StudioAI Plan Recommendation Model</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [4]:
import json
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage

In [ ]:
role = get_execution_role()
sagemaker_session = sage.Session()
boto3 = sagemaker_session.boto_session
bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
s3 = boto3.client("s3")
runtime = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [ ]:
model_name = "plan-recommendation-model"
content_type = "application/json"
real_time_inference_instance_type = "ml.t2.medium"
batch_transform_inference_instance_type = "ml.m5.large"

### A. Create an endpoint

In [ ]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

Once endpoint has been created, you would be able to perform real-time inference.

### B. Prepare input file for performing real-time inference

In [ ]:
# Sample input data
input_data = {
    "data_source": "app/data.csv", # or s3 bucket url to the data s3://plans-recommendation/datasources/data.csv
    "state": "CA",
    "lob": "Small Group",
    "census": 10,
    "sic": "1540",
    "top_n": 50
}

<Add code snippet that shows the payload contents>

### C. Perform real-time inference

In [ ]:
# Invoke SageMaker endpoint
response = runtime.invoke_endpoint(
    EndpointName=model_name, ContentType=content_type, Body=json.dumps(input_data)
)

### D. Visualize output

In [ ]:
# Deserialize the response
response_body = response['Body'].read().decode('utf-8')
response_json = json.loads(response_body)

# Get predictions
print(response_json)

### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [ ]:
# upload the batch-transform job input files to S3
transform_input_folder = "plans-recommendation-model-transform-input"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name)
print("Transform input uploaded to " + transform_input)

In [ ]:
# Run the batch-transform job
transformer = model.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

In [ ]:
# output is available on following path
transformer.output_path

## 4. Clean-up

### A. Delete the model

In [ ]:
model.delete_model()

### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

